In [1]:
import itertools
import pandas as pd
import nltk
import experiment_util

renaming_gold = {'essay_id': 'id', 'prediction_string': 'predictionstring'}
renaming_pred = {'essay_id': 'id', 'prediction_string': 'predictionstring', 'discourse_type': 'class'}

gold_df = pd.read_csv("merged.csv")
gold_df.rename(columns=renaming_gold, inplace=True)
prediction_df = pd.read_csv("train1test2_prediction.csv")
prediction_df.rename(columns=renaming_pred, inplace=True)

In [2]:
clusters_df = pd.read_csv('clusters_effectivness.csv').set_index('id')
gold_df = gold_df.join(clusters_df, on='id')

In [3]:
gold_df_effective = gold_df[gold_df.discourse_effectiveness == 'Effective']
gold_df_effective.to_csv('gold_effective.csv')
gold_df_adequate = gold_df[gold_df.discourse_effectiveness == 'Adequate']
gold_df_adequate.to_csv('gold_adequate.csv')
gold_df_ineffective = gold_df[gold_df.discourse_effectiveness == 'Ineffective']
gold_df_ineffective.to_csv('gold_ineffective.csv')

In [4]:
experiment_util.model_evaluate(prediction_df, gold_df)[1]

Overall evaluation: 0.5544623497603323


{'Lead': {'TP': 1673,
  'FP': 699,
  'FN': 619,
  'Precision': 0.7299301919720768,
  'Recall': 0.7053119730185498,
  'F1': 0.717409948542024},
 'Position': {'TP': 1965,
  'FP': 1104,
  'FN': 2060,
  'Precision': 0.48819875776397514,
  'Recall': 0.6402737047898338,
  'F1': 0.5539892867211729},
 'Claim': {'TP': 4118,
  'FP': 4255,
  'FN': 7860,
  'Precision': 0.34379696109534147,
  'Recall': 0.49181894183685654,
  'F1': 0.4046975578595646},
 'Evidence': {'TP': 7321,
  'FP': 4649,
  'FN': 4785,
  'Precision': 0.6047414505204031,
  'Recall': 0.6116123642439432,
  'F1': 0.6081575012460542},
 'Counterclaim': {'TP': 836,
  'FP': 931,
  'FN': 938,
  'Precision': 0.47125140924464487,
  'Recall': 0.4731182795698925,
  'F1': 0.4721829991527817},
 'Rebuttal': {'TP': 460,
  'FP': 614,
  'FN': 785,
  'Precision': 0.36947791164658633,
  'Recall': 0.42830540037243947,
  'F1': 0.39672272531263475},
 'Concluding Statement': {'TP': 2553,
  'FP': 1108,
  'FN': 799,
  'Precision': 0.7616348448687351,
  'Re

In [5]:
experiment_util.model_evaluate(prediction_df.copy(), gold_df_ineffective.copy())[1]

Overall evaluation: 0.1195244716633719


{'Rebuttal': {'TP': 60,
  'FP': 1014,
  'FN': 152,
  'Precision': 0.2830188679245283,
  'Recall': 0.055865921787709494,
  'F1': 0.09331259720062209},
 'Evidence': {'TP': 1610,
  'FP': 10356,
  'FN': 1547,
  'Precision': 0.5099778270509978,
  'Recall': 0.13454788567608222,
  'F1': 0.2129207167889969},
 'Claim': {'TP': 457,
  'FP': 7878,
  'FN': 1019,
  'Precision': 0.30962059620596205,
  'Recall': 0.05482903419316137,
  'F1': 0.09316073794720212},
 'Counterclaim': {'TP': 70,
  'FP': 1697,
  'FN': 136,
  'Precision': 0.33980582524271846,
  'Recall': 0.039615166949632144,
  'F1': 0.07095793208312215},
 'Position': {'TP': 140,
  'FP': 2929,
  'FN': 331,
  'Precision': 0.29723991507430997,
  'Recall': 0.045617464972303685,
  'F1': 0.07909604519774012},
 'Lead': {'TP': 186,
  'FP': 2186,
  'FN': 179,
  'Precision': 0.5095890410958904,
  'Recall': 0.07841483979763912,
  'F1': 0.1359152356594812},
 'Concluding Statement': {'TP': 321,
  'FP': 3340,
  'FN': 261,
  'Precision': 0.5515463917525774

In [6]:
experiment_util.model_evaluate(prediction_df.copy(), gold_df_adequate.copy())[1]

Overall evaluation: 0.4020783330457223


{'Lead': {'TP': 886,
  'FP': 1486,
  'FN': 359,
  'Precision': 0.7116465863453815,
  'Recall': 0.37352445193929174,
  'F1': 0.489908764169201},
 'Position': {'TP': 1394,
  'FP': 1675,
  'FN': 1391,
  'Precision': 0.5005385996409336,
  'Recall': 0.4542196155099381,
  'F1': 0.4762555517594807},
 'Claim': {'TP': 2271,
  'FP': 6083,
  'FN': 4827,
  'Precision': 0.31994928148774304,
  'Recall': 0.27184582236054583,
  'F1': 0.29394253171110535},
 'Evidence': {'TP': 3457,
  'FP': 8508,
  'FN': 2608,
  'Precision': 0.5699917559769168,
  'Recall': 0.28892603426661095,
  'F1': 0.3834719911259013},
 'Counterclaim': {'TP': 497,
  'FP': 1270,
  'FN': 654,
  'Precision': 0.4317984361424848,
  'Recall': 0.2812676853423882,
  'F1': 0.3406442769019877},
 'Concluding Statement': {'TP': 1554,
  'FP': 2107,
  'FN': 392,
  'Precision': 0.7985611510791367,
  'Recall': 0.42447418738049714,
  'F1': 0.5543071161048689},
 'Rebuttal': {'TP': 244,
  'FP': 830,
  'FN': 450,
  'Precision': 0.3515850144092219,
  'Re

In [7]:
experiment_util.model_evaluate(prediction_df.copy(), gold_df_effective.copy())[1]

Overall evaluation: 0.275240039779121


{'Lead': {'TP': 601,
  'FP': 1771,
  'FN': 83,
  'Precision': 0.8786549707602339,
  'Recall': 0.2533726812816189,
  'F1': 0.3933246073298429},
 'Position': {'TP': 431,
  'FP': 2638,
  'FN': 340,
  'Precision': 0.5590142671854734,
  'Recall': 0.14043662430759205,
  'F1': 0.22447916666666667},
 'Evidence': {'TP': 2254,
  'FP': 9711,
  'FN': 632,
  'Precision': 0.781011781011781,
  'Recall': 0.18838278311742582,
  'F1': 0.30354858258703116},
 'Claim': {'TP': 1390,
  'FP': 6958,
  'FN': 2016,
  'Precision': 0.40810334703464474,
  'Recall': 0.16650694777192143,
  'F1': 0.236515228858261},
 'Concluding Statement': {'TP': 678,
  'FP': 2983,
  'FN': 148,
  'Precision': 0.8208232445520581,
  'Recall': 0.18519530183010108,
  'F1': 0.3022063739692445},
 'Counterclaim': {'TP': 269,
  'FP': 1498,
  'FN': 150,
  'Precision': 0.6420047732696897,
  'Recall': 0.1522354272778721,
  'F1': 0.24611161939615736},
 'Rebuttal': {'TP': 156,
  'FP': 918,
  'FN': 185,
  'Precision': 0.4574780058651026,
  'Recall

In [8]:
def argument_similarity(arg1, arg2):
    overlap = experiment_util.calc_overlap2(arg1, arg2)
    return (overlap[0] + overlap[1]) / 2

def mean_arg_similarity_per_selection(gold_df, selector = 'cluster', scnd_gold_df = None):
    mean_similarities = dict()
    overall_sum_similarity = 0
    overall_comb_cnt = 0
    for selection in list(set(gold_df[selector].values)):    # for each cluster/ label
        gold_df_selection = gold_df[gold_df[selector] == selection]
        dtext_tokens = [set(nltk.tokenize.word_tokenize(dt.lower())) for dt in gold_df_selection['discourse_text']]

        if scnd_gold_df is not None:
            gold_df_selection2 = scnd_gold_df[scnd_gold_df[selector] == selection]
            dtext_tokens2 = [set(nltk.tokenize.word_tokenize(dt.lower())) for dt in gold_df_selection2['discourse_text']]
            dt_combinations = list(itertools.product(dtext_tokens, dtext_tokens2))
        else:
            dt_combinations = list(itertools.combinations(dtext_tokens, 2))

        mean_similarities[selection] = 0
        for dt_combination in dt_combinations:
            arg_similarity = argument_similarity(*dt_combination)
            mean_similarities[selection] += arg_similarity
            overall_sum_similarity += arg_similarity
        mean_similarities[selection] /= len(dt_combinations)
        overall_comb_cnt += len(dt_combinations)

    print("⌀ =", sum(mean_similarities.values()) / len(mean_similarities.values()))
    print("⌀ w =", overall_sum_similarity / overall_comb_cnt)
    return mean_similarities


In [9]:
mean_arg_similarity_per_selection(gold_df_effective, 'cluster')

⌀ = 0.24430127782811323
⌀ w = 0.24560972857694355


{0: 0.25169242022060945,
 1: 0.25455782861493287,
 2: 0.25313766733720355,
 3: 0.2583747008743779,
 4: 0.2352838340681213,
 5: 0.24758143377968708,
 6: 0.26200629435921086,
 7: 0.22942226953977315,
 8: 0.25640722261962584,
 9: 0.23277311562023165,
 10: 0.23950458445840042,
 11: 0.22229059349007965,
 12: 0.22813990359879965,
 13: 0.23625972233355036,
 14: 0.2570875765070949}

In [10]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'cluster')

⌀ = 0.2053458242229327
⌀ w = 0.21246518520864516


{0: 0.22362750001684178,
 1: 0.20671682670867264,
 2: 0.21620518637692557,
 3: 0.20809666035516625,
 4: 0.21523235890972903,
 5: 0.16571891234235164,
 6: 0.20284364402384214,
 7: 0.2235217319625553,
 8: 0.223254586060867,
 9: 0.19420059497079784,
 10: 0.19210338466929042,
 11: 0.21177802592119369,
 12: 0.16811738530463557,
 13: 0.20704031002808967,
 14: 0.2217302556930319}

In [11]:
mean_arg_similarity_per_selection(gold_df_effective, 'cluster', gold_df_ineffective)

⌀ = 0.216602131053382
⌀ w = 0.22288490337981065


{0: 0.22794852998568868,
 1: 0.21828028333334443,
 2: 0.22990613182679245,
 3: 0.2232937745937118,
 4: 0.21860529749103105,
 5: 0.20588865310919355,
 6: 0.22461596799223293,
 7: 0.21524198178902512,
 8: 0.23190302636508775,
 9: 0.19423434632797623,
 10: 0.21320458071064924,
 11: 0.2054959614176357,
 12: 0.1928133395688812,
 13: 0.21462573370564986,
 14: 0.23297435758383048}

In [12]:
mean_arg_similarity_per_selection(gold_df_effective, 'discourse_type')

⌀ = 0.2052113746886755
⌀ w = 0.18915669370052202


{'Position': 0.17996050112386375,
 'Evidence': 0.239732057446455,
 'Counterclaim': 0.18702506193153942,
 'Lead': 0.22650087746831546,
 'Rebuttal': 0.20682238842689385,
 'Concluding Statement': 0.24822674068066608,
 'Claim': 0.14821199574299493}

In [13]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'discourse_type')

⌀ = 0.1628915935224877
⌀ w = 0.18932930134348794


{'Evidence': 0.20383239273108977,
 'Counterclaim': 0.1352833438207233,
 'Lead': 0.16041218362519372,
 'Rebuttal': 0.1761502677411512,
 'Concluding Statement': 0.19103374095322137,
 'Position': 0.14313648827500547,
 'Claim': 0.13039273751102903}

In [14]:
mean_arg_similarity_per_selection(gold_df_effective, 'discourse_type', gold_df_ineffective)

⌀ = 0.1770725331138979
⌀ w = 0.18665736111294243


{'Position': 0.14952063127387055,
 'Evidence': 0.2175248218370154,
 'Counterclaim': 0.15257677447170728,
 'Lead': 0.19207349944384555,
 'Rebuttal': 0.18196125406175148,
 'Concluding Statement': 0.2147805797090662,
 'Claim': 0.1310701710000289}

In [15]:
list(itertools.combinations([1,2,3,4], 2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]